In [ ]:
!pip install retrying

In [ ]:
### Set your API Key

%set_env QI_API_KEY = YOUR_API_KEY

In [ ]:
### Download Qi_wrapper from https://github.com/Quant-Insight/Qi_API

import Qi_wrapper
import pandas
import numpy as np

In [ ]:
### Variables

benchmark_name = 'SPX'
date = '2020-03-16'
term = 'Long Term'

In [ ]:
### Create Portfolio with positions and weights

portfolio = pandas.DataFrame({'Name':['MSFT','GOOG','PG','JPM','AAPL','FB','GS','MRO','DEI','SO'],
            'Position':[194300000,142000000,122440000,122440000,113940000,109690000,106290000,102040000,102040000,99060000]})
                  
portfolio['Weight'] = [abs(x)/sum(abs(portfolio.Position)) for x in portfolio.Position]
portfolio['L/S'] = [1]*len(portfolio)

In [ ]:
### Creates cash exposure table

Qi_wrapper.get_portfolio_cash_exposures_bucket(portfolio,date)

In [ ]:
### Gets portfolio sensitivites to top 15 macro factors

sens = Qi_wrapper.get_portfolio_sens_exposures_factor(portfolio,date)
sens[abs(sens.loc['Total']).nlargest(10).index].loc[['Total']]

In [ ]:
### Gets the std's of each of the top 15 macro factors

Qi_wrapper.get_factor_stdevs(benchmark_name,date,term).T[abs(sens.loc['Total']).nlargest(10).index]

In [ ]:
### Create pie chart data

import numpy as np

sens_buckets = Qi_wrapper.get_portfolio_sens_exposures_bucket(portfolio,date)
pie_data = sens_buckets.loc[['Total']].T.sort_values(by='Total')[::-1]
pie_data.Total = [abs(x)/abs(pie_data).Total.sum() for x in pie_data.Total]

In [ ]:
### Plot pie chart

import matplotlib.pyplot as plt


labels = pie_data.index
sizes = pie_data.Total

fig1, ax1 = plt.subplots(figsize=(15, 12))


ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=False, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
### Creates data for bar and scatter plot (Risk relative to benchmark)

bar_plot_data = sens_buckets.loc[['Total']].T.sort_values(by='Total')[::-1]

benchmark = Qi_wrapper.get_bucket_drivers(benchmark_name,date,term)

idxs = bar_plot_data.index.tolist()

bar_plot_data[benchmark_name] = benchmark[idxs].T.Sensitivity

In [ ]:
### Plot sensitivities relative to a benchmark

fig = plt.figure(figsize=(15, 8))
ax = fig.add_subplot(111)
ax.spines["top"].set_visible(False)  
ax.spines["right"].set_visible(False)
ax.bar(bar_plot_data.index,bar_plot_data.Total,zorder=1)
ax.scatter(bar_plot_data.index,bar_plot_data.SPX,zorder=2,color='red')
plt.xticks(rotation='vertical')
plt.ylabel('% Chng in Portfolio for a 1std move in Factor',size=12)
plt.legend([benchmark_name,'Fund'])
plt.show()